In [68]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpStatus, LpMinimize

# Carregar o arquivo Excel
file_path = 'C:/Users/bruno/Desktop/TCC Bruno/Projeto.xlsx'
df = pd.read_excel(file_path, sheet_name='tabela')

# Definir o problema
model = LpProblem(name="modelo_nutricional", sense=LpMinimize)

# Configurar variáveis de decisão para cada alimento, limitado a 100g exceto arroz e feijão
quantidades = {row['Alimentos']: LpVariable(name=row['Alimentos'], lowBound=0, upBound=100) 
               for _, row in df.iterrows() if row['Alimentos'] not in ['Arroz', 'Feijão']}

# Adicionar restrições para arroz e feijão
quantidades['Arroz'] = LpVariable(name='Arroz', lowBound=50, upBound=100)
quantidades['Feijão'] = LpVariable(name='Feijão', lowBound=50, upBound=100)

# Restrições nutricionais
proteinas = lpSum([quantidades[alimento] * (df.loc[df['Alimentos'] == alimento, 'Proteina (100gr)'].values[0] / 100)
                   for alimento in quantidades]) == 137
lipideos = lpSum([quantidades[alimento] * (df.loc[df['Alimentos'] == alimento, 'Lipídeos (100gr)'].values[0] / 100)
                  for alimento in quantidades]) == 61
carboidratos = lpSum([quantidades[alimento] * (df.loc[df['Alimentos'] == alimento, 'Carboidratos (100gr)'].values[0] / 100)
                      for alimento in quantidades]) == 412
fibras = lpSum([quantidades[alimento] * (df.loc[df['Alimentos'] == alimento, 'Fibras (100gr)'].values[0] / 100)
                for alimento in quantidades]) == 69

model += proteinas
model += lipideos
model += carboidratos
model += fibras

# Restrição de orçamento diário
custo_diario = lpSum([quantidades[alimento] * (df.loc[df['Alimentos'] == alimento, 'Custo R$ (100gr)'].values[0] / 100)
                      for alimento in quantidades]) <= 26.63

model += custo_diario

# Função objetivo (minimizar o custo)
model += lpSum([quantidades[alimento] * (df.loc[df['Alimentos'] == alimento, 'Custo R$ (100gr)'].values[0] / 100)
                for alimento in quantidades])

# Função para calcular os nutrientes totais baseados nas quantidades escolhidas
def calcular_nutrientes(df, quantidades):
    proteinas = sum([quantidades[alimento].value() * (df.loc[df['Alimentos'] == alimento, 'Proteina (100gr)'].values[0] / 100)
                     for alimento in quantidades])
    lipideos = sum([quantidades[alimento].value() * (df.loc[df['Alimentos'] == alimento, 'Lipídeos (100gr)'].values[0] / 100)
                    for alimento in quantidades])
    carboidratos = sum([quantidades[alimento].value() * (df.loc[df['Alimentos'] == alimento, 'Carboidratos (100gr)'].values[0] / 100)
                        for alimento in quantidades])
    fibras = sum([quantidades[alimento].value() * (df.loc[df['Alimentos'] == alimento, 'Fibras (100gr)'].values[0] / 100)
                  for alimento in quantidades])
    return proteinas, lipideos, carboidratos, fibras

# Resolver o problema com as restrições e objetivos definidos anteriormente
model.solve()

# Obter as quantidades ótimas de cada alimento
quantidades_otimas = {alimento: quantidades[alimento].value() for alimento in quantidades}

# Filtrar apenas os alimentos escolhidos
alimentos_escolhidos = {alimento: quantidade for alimento, quantidade in quantidades_otimas.items() if quantidade > 0}

# Calcular o custo diário total
custo_diario_total = sum(quantidades[alimento].value() * (df.loc[df['Alimentos'] == alimento, 'Custo R$ (100gr)'].values[0] / 100)
                         for alimento in alimentos_escolhidos)

# Calcular o custo mensal total
custo_mensal_total = custo_diario_total * 30

# Criar DataFrame para os alimentos escolhidos
df_alimentos_escolhidos = pd.DataFrame(alimentos_escolhidos.items(), columns=["Alimento", "Quantidade (g)"])
df_alimentos_escolhidos['Custo Diário (R$)'] = df_alimentos_escolhidos['Alimento'].apply(lambda x: quantidades[x].value() * (df.loc[df['Alimentos'] == x, 'Custo R$ (100gr)'].values[0] / 100))

# Calcular os macronutrientes totais
proteinas_totais, lipideos_totais, carboidratos_totais, fibras_totais = calcular_nutrientes(df, quantidades)

# Adicionar os custos totais e macronutrientes ao DataFrame
df_alimentos_escolhidos.loc['Total'] = ['Total', df_alimentos_escolhidos['Quantidade (g)'].sum(), custo_diario_total]
df_macros = pd.DataFrame({
    'Macronutriente': ['Proteínas', 'Lipídeos', 'Carboidratos', 'Fibras'],
    'Total (g)': [proteinas_totais, lipideos_totais, carboidratos_totais, fibras_totais]
})

# Melhorar a exibição dos resultados
print("Status do modelo:", LpStatus[model.status])
print("\nCusto Diário Total (R$): {:.2f}".format(custo_diario_total))
print("Custo Mensal Total (R$): {:.2f}".format(custo_mensal_total))
print("\nAlimentos Escolhidos e Quantidades:")
print(df_alimentos_escolhidos[['Alimento', 'Quantidade (g)', 'Custo Diário (R$)']])
print("\nMacronutrientes Totais:")
print(df_macros)


Status do modelo: Optimal

Custo Diário Total (R$): 12.16
Custo Mensal Total (R$): 364.66

Alimentos Escolhidos e Quantidades:
                    Alimento  Quantidade (g)  Custo Diário (R$)
0                      Aveia      100.000000           1.780000
1                   Mandioca      100.000000           0.340000
2      Pão de Forma Integral      100.000000           0.990000
3                Pão Francês       55.105012           0.826575
4                     Figado      100.000000           1.490000
5                      Lombo       34.254783           0.664543
6            Peito de Frango      100.000000           1.890000
7        Sobrecoxa de Frango      100.000000           0.980000
8              Banana Nanica      100.000000           0.390000
9                    Linhaça       74.060097           1.096089
10                   Abobora       20.660166           0.070245
11               Batata Doce      100.000000           0.460000
12                     Arroz      100.000